In [1]:
import numpy as np
import pandas as pd

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [5]:
df = pd.read_csv('Loan_default.csv')
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


In [11]:
df.drop(columns = ['LoanID', 'MonthsEmployed', 'NumCreditLines', 'Education', 'EmploymentType', 'MaritalStatus', 'HasCoSigner'], inplace = True)

In [13]:
df['HasMortgage'] = df['HasMortgage'].replace(['Yes', 'No'], [1, 0])
df['HasDependents'] = df['HasDependents'].replace(['Yes', 'No'], [1, 0])
df['LoanPurpose'] = df['LoanPurpose'].replace(['Business', 'Home', 'Education', 'Other', 'Auto'], [0, 1, 2, 3, 4])

C:\Users\dipan\AppData\Local\Temp\ipykernel_13364\1617550533.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HasMortgage'] = df['HasMortgage'].replace(['Yes', 'No'], [1, 0])
C:\Users\dipan\AppData\Local\Temp\ipykernel_13364\1617550533.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['HasDependents'] = df['HasDependents'].replace(['Yes', 'No'], [1, 0])
C:\Users\dipan\AppData\Local\Temp\ipykernel_13364\1617550533.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future versio

In [17]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [19]:
df[['Age', 'Income', 'LoanAmount', 'CreditScore', 'InterestRate', 'LoanTerm', 'DTIRatio']] = ss.fit_transform(df[['Age', 'Income', 'LoanAmount', 'CreditScore', 'InterestRate', 'LoanTerm', 'DTIRatio']])

In [23]:
df

,Age,Income,LoanAmount,CreditScore,InterestRate,LoanTerm,DTIRatio,HasMortgage,HasDependents,LoanPurpose,Default
0,0.833990,0.089693,-1.086833,-0.341492,0.261771,-0.001526,-0.260753,1,1,3,0
1,1.701221,-0.823021,-0.044309,-0.731666,-1.308350,1.412793,0.778585,0,0,3,0
2,0.166888,0.043854,0.022715,-0.775718,1.156831,-0.708685,-0.823728,1,1,4,1
3,-0.767053,-1.303452,-1.168538,1.061875,-0.967805,-0.708685,-1.170174,0,0,0,0
4,1.100830,-1.592855,-1.671921,0.369631,-1.052188,0.705634,0.995114,0,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...
255342,-1.634285,-1.142632,1.173101,-0.209337,0.093006,-1.415845,1.514783,0,0,3,0
255343,-0.767053,-0.783984,0.879724,-0.398130,-0.292744,-0.708685,-1.256785,0,0,1,1
255344,0.833990,0.059562,1.139391,0.143078,-1.236022,1.412793,-0.000918,1,1,4,0
255345,-0.099952,0.066979,-0.945840,1.477221,1.116146,0.705634,-0.260753,1,1,3,0


In [25]:
x = df.iloc[:,:10]
y = df.iloc[:, -1]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [35]:
class CreditRiskNN(nn.Module):
    def __init__(self, input_size):
        super(CreditRiskNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)  # First hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)  # Second hidden layer
        self.fc3 = nn.Linear(32, 1)  # Output layer
        self.sigmoid = nn.Sigmoid()  # Sigmoid for binary classification

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

In [47]:
# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Define model, loss function, and optimizer
input_size = X_train.shape[1]
model = CreditRiskNN(input_size)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 500
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [10/500], Loss: 0.5808
Epoch [20/500], Loss: 0.4763
Epoch [30/500], Loss: 0.3983
Epoch [40/500], Loss: 0.3632
Epoch [50/500], Loss: 0.3558
Epoch [60/500], Loss: 0.3496
Epoch [70/500], Loss: 0.3433
Epoch [80/500], Loss: 0.3391
Epoch [90/500], Loss: 0.3358
Epoch [100/500], Loss: 0.3334
Epoch [110/500], Loss: 0.3317
Epoch [120/500], Loss: 0.3305
Epoch [130/500], Loss: 0.3297
Epoch [140/500], Loss: 0.3292
Epoch [150/500], Loss: 0.3289
Epoch [160/500], Loss: 0.3286
Epoch [170/500], Loss: 0.3283
Epoch [180/500], Loss: 0.3281
Epoch [190/500], Loss: 0.3278
Epoch [200/500], Loss: 0.3275
Epoch [210/500], Loss: 0.3272
Epoch [220/500], Loss: 0.3269
Epoch [230/500], Loss: 0.3266
Epoch [240/500], Loss: 0.3263
Epoch [250/500], Loss: 0.3260
Epoch [260/500], Loss: 0.3257
Epoch [270/500], Loss: 0.3255
Epoch [280/500], Loss: 0.3252
Epoch [290/500], Loss: 0.3250
Epoch [300/500], Loss: 0.3247
Epoch [310/500], Loss: 0.3245
Epoch [320/500], Loss: 0.3243
Epoch [330/500], Loss: 0.3241
Epoch [340/500], Lo

In [49]:
# Convert test data to tensors
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Get predictions
with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = predictions.numpy()  # Convert to NumPy array
    predictions = (predictions > 0.5).astype(int)  # Convert probabilities to 0 or 1

# Calculate accuracy
from sklearn.metrics import accuracy_score
print("Model Accuracy:", accuracy_score(y_test, predictions))


Model Accuracy: 0.8860583512825534
